# StyleAdv attack

Prerequisites: You need to add the pre-trained [style transfer models](https://drive.google.com/drive/folders/12ImHH2kJKw1Vs3rDUSRytP3DZYcHdsZw?usp=sharing) (by https://github.com/martiansideofthemoon/style-transfer-paraphrase) to your Google Drive.

Add a shortcut to your Google Drive under /My Drive/style_transfer_paraphrase

In [1]:
!git clone https://github.com/yifengd/StyleAttack.git
!pip install transformers >> /dev/null


Cloning into 'StyleAttack'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 388 (delta 3), reused 0 (delta 0), pack-reused 380
Receiving objects: 100% (388/388), 58.35 MiB | 18.98 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [2]:
!git -C /content/StyleAttack pull

Already up to date.


In [3]:
# Mount on Google Drive File System
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/StyleAttack/experiments

# INPUT_PATH = "../data/clean/ag/test_100.tsv"
INPUT_PATH = "/content/drive/MyDrive/AdversarialXAI/Raw\ data/sst2_val_100.tsv"

# MODEL_NAME = "textattack/distilbert-base-uncased-imdb"
MODEL_NAME = "textattack/distilbert-base-uncased-SST-2" # Note this model is on huggingface but not listed on the textattack website

cmd = " ".join(["python",
              "attack.py",
              "--model_name", MODEL_NAME,
              "--bert_type", "distilbert-base-cased",
              "--orig_file_path", INPUT_PATH,
              "--model_dir", "/content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large",
              "--output_file_path", "/content/record_100.log"
])

!{cmd}


/content/StyleAttack/experiments
Downloading: 100% 486/486 [00:00<00:00, 651kB/s]
Downloading: 100% 255M/255M [00:06<00:00, 39.8MB/s]
Downloading: 100% 29.0/29.0 [00:00<00:00, 34.9kB/s]
Downloading: 100% 411/411 [00:00<00:00, 450kB/s]
Downloading: 100% 208k/208k [00:00<00:00, 1.80MB/s]
Downloading: 100% 426k/426k [00:00<00:00, 2.95MB/s]


In [ ]:

# !CUDA_VISIBLE_DEVICES=0 python attack.py --model_name textattack/distilbert-base-uncased-ag-news --bert_type distilbert-base-uncased --orig_file_path ../data/clean/ag/test_100.tsv --model_dir /content/drive/MyDrive/style_transfer_paraphrase/models/paraphraser_gpt2_large --output_file_path /content/record_100.log


In [ ]:
import pandas as pd

# dfs = []

# for label in label_set:
#   out_file = f'/content/record_100_{label}.log'
#   df = pd.read_table(out_file,sep='\t')
#   dfs.append(df)

# df = pd.concat(dfs, ignore_index=True)

# df

In [ ]:
out_file = f'/content/record_100.log'
df = pd.read_table(out_file,sep='\t')

print(df.columns)

df = df.rename(columns={" orig_sent ": "original_text", " adv_sent ": "adversarial_text", 'p_val ': 'p_val', ' labels': 'labels'})
print(df.columns)

print(len(df))

df


Index(['p_val ', ' orig_sent ', ' adv_sent ', ' labels'], dtype='object')
Index(['p_val', 'original_text', 'adversarial_text', 'labels'], dtype='object')
93


,p_val,original_text,adversarial_text,labels
0,-1,"1891: Stalwart, morally upright military doct...","1891: Stalwart, morally upright military doct...",1
1,-1,"Robert Cummings, Laraine Day and Jean Muir st...","Robert Cummings, Laraine Day and Jean Muir st...",1
2,-1,"When Precious Bane aired in America, I was st...","When Precious Bane aired in America, I was st...",1
3,1,This film is terribly bad. Kevin Spacey is a ...,"he's really bad, but he's really good at LA C...",0
4,1,OK i have seen Hershall Gordon Lewis movies b...,"I've seen Hershall Gordon Lewis films before,...",0
...,...,...,...,...
88,-1,I will not spoil your surprise by mentioning ...,I will not spoil your surprise by mentioning ...,0
89,1,"A gem from Japan, where so many of the world'...","in the film, which is being made by the best ...",1
90,-1,"From the opening drum hit of ""We're Gonna Gro...","From the opening drum hit of ""We're Gonna Gro...",1
91,-1,"Not since J. Michael Straczynski's Babylon 5,...","Not since J. Michael Straczynski's Babylon 5,...",1


In [ ]:
# df.query("p_val == 1")

In [ ]:
import os

out_name = "sst2_val_100_styleadv_distilbert.csv"

out_dir = '/content/drive/MyDrive/AdversarialXAI/Adversarial Samples/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

df.to_csv(os.path.join(out_dir, out_name), index=False)